![ROOT Logo](http://root.cern.ch/img/logos/ROOT_Logo/website-banner/website-banner-%28not%20root%20picture%29.jpg)
<br />
# **Distributing a TOTEM analysis with RDataFrame in Python**
<hr style="border-top-width: 4px; border-top-color: #34609b;">

Code shown in this notebook has been extracted from a real TOTEM analysis. Please refer to the [original source](https://github.com/jan-kaspar/analysis_elastic.6500GeV.beta90.10sigma) for further information.

---

### Full Analysis

Note that, unlike [DistillDistibution.ipynb](https://github.com/JavierCVilla/RDataFrame-Totem/blob/master/DistillDistibution.ipynb), this notebook has been modify to run the analysis on the **full dataset (4.7TB of ROOT Ntuple events)**.


### Introduction

The aim of this notebook is to present a complex real analysis rewritten from ROOT C++ to [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html) which is able to run distributed using Spark as the backend framework.

For the sake of completeness, the original code is splitted in two scripts with a clear purpose each:

- `distill.cc`: 
    + Select branches to read
    + Apply first filter to remove invalid data, thereby reducing extensively the amount of events to be processed
    + Create a new ROOT file out of the reduced dataset  
    + Long execution time $\mathcal{O}(\text{hours})$, hence is meant to be run once

- `distributions.cc`: (to be run multiple times while tweaking parameters)
    + Read branches from the root file created by distill.cc
    + Apply multiple filters
    + Produce TH1D, TH2D, TProfiles and TGraphs
    + Save all results to a ROOT file 
    + Much shorter execution time, so it can be run multiple times while tunning parameters of the analysis.

In this notebook, parts of the RDataFrame version of the code have been rearranged in order to run distributed through Spark. An extended version of this RDataFrame code is [hosted here](https://github.com/JavierCVilla/RDataFrame-Totem).


### Running the analysis

First import the necessary packages. We need the `DistROOT` package to help us distribute the `RDataFrame` computation with **Spark**.

In [1]:
import ROOT
from ROOT import TCanvas
from DistROOT import DistTree

Welcome to JupyROOT 6.17/01


Next four cells include the code which will read, filter and produce histograms with the analysis results.

In [2]:
# Branches clasified by diagonal
diagonals = {
    # return a tuple: ([left] verticals in 45, [right] verticals in 56))
    "d45b_56t"  : (["track_rp_5", "track_rp_21", "track_rp_25"],
                   ["track_rp_104", "track_rp_120", "track_rp_124"]),
    "ad45b_56b" : (["track_rp_5", "track_rp_21", "track_rp_25"],
                   ["track_rp_105", "track_rp_121", "track_rp_125"]),
    "d45t_56b"  : (["track_rp_4", "track_rp_20", "track_rp_24"],
                   ["track_rp_105", "track_rp_121", "track_rp_125"]),
    "ad45t_56t" : (["track_rp_4", "track_rp_20", "track_rp_24"],
                   ["track_rp_104", "track_rp_120", "track_rp_124"])
}

# Data source files
DS = {
    'DS1'   : '4495', 
    'DS2'   : '4496', 
    'DS3'   : '4499', 
    'DS4'   : '4505', 
    'DS5'   : '4509', 
    'DS6'   : '4510', 
    'DS7'   : '4511'
}

# Select branches
selected_diagonal = "d45b_56t"

rp_left, rp_right = diagonals[selected_diagonal]

# Columns per branch
attributes = ['valid', 'x', 'y']

full_branches = ["{}.{}".format(c,a) for a in attributes for c in rp_left+rp_right ]

# Split left and right branch on valid, x and y
valids = [ "(unsigned int) {}".format(v) for v in full_branches[0:6]]
xs     = full_branches[6:12]
ys     = full_branches[12:18]


print("Selected branches: \n" + "\n\t".join(full_branches))

# Filter to select valid branches
filter_code = """({0}.valid + {1}.valid + {2}.valid ) >= 2 &&
({3}.valid + {4}.valid + {5}.valid ) >= 2
""".format(*(rp_left+rp_right))

Selected branches: 
track_rp_5.valid
	track_rp_21.valid
	track_rp_25.valid
	track_rp_104.valid
	track_rp_120.valid
	track_rp_124.valid
	track_rp_5.x
	track_rp_21.x
	track_rp_25.x
	track_rp_104.x
	track_rp_120.x
	track_rp_124.x
	track_rp_5.y
	track_rp_21.y
	track_rp_25.y
	track_rp_104.y
	track_rp_120.y
	track_rp_124.y


In [3]:
def distill(rdf):
    # Filter and define output branches
    return rdf.Filter(filter_code)   \
       .Define("v_L_1_F", valids[0]) \
       .Define("v_L_2_N", valids[1]) \
       .Define("v_L_2_F", valids[2]) \
       .Define("v_R_1_F", valids[3]) \
       .Define("v_R_2_N", valids[4]) \
       .Define("v_R_2_F", valids[5]) \
       .Define("x_L_1_F", xs[0]) \
       .Define("x_L_2_N", xs[1]) \
       .Define("x_L_2_F", xs[2]) \
       .Define("x_R_1_F", xs[3]) \
       .Define("x_R_2_N", xs[4]) \
       .Define("x_R_2_F", xs[5]) \
       .Define("y_L_1_F", ys[0]) \
       .Define("y_L_2_N", ys[1]) \
       .Define("y_L_2_F", ys[2]) \
       .Define("y_R_1_F", ys[3]) \
       .Define("y_R_2_N", ys[4]) \
       .Define("y_R_2_F", ys[5]) \
       .Define("timestamp",    "(unsigned int) (event_info.timestamp - 1444860000)") \
       .Define("run_num",      "(unsigned int) event_info.run_no")                 \
       .Define("bunch_num",    "trigger_data.bunch_num")            \
       .Define("event_num",    "trigger_data.event_num")            \
       .Define("trigger_num",  "trigger_data.trigger_num")          \
       .Define("trigger_bits", "trigger_data.input_status_bits")     

In [4]:
# binnings
binnings = ["ub", "ob-1-10-0.2", "ob-1-30-0.2"]

In [5]:
def distributions(rdf):
    
    hlist = []
    
    import ROOT
    #ROOT.ROOT.EnableImplicitMT(2)
    
    # Load C++ headers
    ROOT.gInterpreter.Declare('#include "common_definitions.h"')
    ROOT.gInterpreter.Declare('#include "parameters_global.h"')
    ROOT.gInterpreter.Declare('#include "common_algorithms.h"')
    ROOT.gInterpreter.Declare('#include "parameters.h"')
    ROOT.gInterpreter.Declare('#include "common.h"')
    
    detailsLevel         = 0 # 0: no details, 1: some details, >= 2 all details
    overrideCutSelection = False # whether the default cut selection should be overriden by the command-line selection
    cutSelectionString   = None
    outputDir            = "."
    inputDir             = "."
    input_n_si           = 4.0
    time_group_divisor   = 0
    time_group_remainder = 0
    event_group_divisor  = 0
    event_group_index    = 0
    evIdxStep            = 1
    maxTaggedEvents      = 0
    
    M_PI = 3.14159265358979323846264338328      # Pi
    
    # init diagonal settings
    ROOT.Init("45b_56t")

    # default parameters
    input_n_si = 4.0

    # select cuts
    ROOT.anal.BuildCuts()
    ROOT.anal.n_si = input_n_si

    # alignment init
    for i,_ in  enumerate(ROOT.alignmentSources):
        ROOT.alignmentSources[i].Init()

    # get time-dependent corrections
    corrg_pileup = None
    if ROOT.anal.use_pileup_efficiency_fits:
        path = inputDir + "/pileup_fit_combined.root"
        puF = ROOT.TFile.Open(path)
        if not os.path.exists(puF):
            print("ERROR: pile-up correction file `%s' cannot be opened.\n" % path);
        if diagonal == "d45b_56t":
            #corrg_pileup = (TGraph *) puF.Get("45b_56t/dgn");
            corrg_pileup = puF.Get("45b_56t/dgn")
        if diagonal == "d45t_56b":
            #corrg_pileup = (TGraph *) puF.Get("45b_56t/dgn");
            corrg_pileup = puF.Get("45t_56b/dgn")

    # get th_y* dependent efficiency correction
    f_3outof4_efficiency_L_F = None
    f_3outof4_efficiency_L_N = None
    f_3outof4_efficiency_R_N = None
    f_3outof4_efficiency_R_F = None

    if ROOT.anal.use_3outof4_efficiency_fits:
        path = inputDir + "/eff3outof4_details_fit_old.root"
        effFile = ROOT.TFile.Open(path)
        if (os.path.exists(effFile)):
            print("ERROR: 3-out-of-4 efficiency file `%s' cannot be opened.\n" % path)

        diagonal = selected_diagonal;
        f_3outof4_efficiency_L_F = effFile.Get(diagonal + "/L_F/fit")
        f_3outof4_efficiency_L_N = effFile.Get(diagonal + "/L_N/fit")
        f_3outof4_efficiency_R_N = effFile.Get(diagonal + "/R_N/fit")
        f_3outof4_efficiency_R_F = effFile.Get(diagonal + "/R_F/fit")
        
        print("\n>> using 3-out-of-4 fits: %s, %s, %s, %s\n" % (f_3outof4_efficiency_L_F, 
                                                                f_3outof4_efficiency_L_N, 
                                                                f_3outof4_efficiency_R_N, 
                                                                f_3outof4_efficiency_R_F))

    # book metadata histograms
    ROOT.timestamp_bins = ROOT.timestamp_max - ROOT.timestamp_min + 1.

    # Create bh_t_* hists
    bh_t_Nev_before = dict()
    bh_t_Nev_after_no_corr = dict()
    bh_t_before = dict()
    bh_t_after = dict()
    bh_t_after_no_corr = dict()
    bp_t_phi_corr = dict()
    bp_t_full_corr = dict()

    binning_setup = dict()
    for b in binnings:
        binning = ROOT.BuildBinningRDF(ROOT.anal, b)
        binning_setup[b] = binning

    # zero counters
    n_ev_full = 0;
    n_ev_cut = dict();
    for ci in range(ROOT.anal.N_cuts):
        n_ev_cut[ci] = 0

    th_min = 1E100;
    th_y_L_min = +1E100; th_y_R_min = +1E100

    N_anal=0; N_anal_zeroBias=0;
    N_zeroBias_el=0; N_zeroBias_el_RP_trig=0;
    N_4outof4=0; N_el=0;
    N_el_T2trig=0; N_4outof4_T2trig=0;
    N_el_raw=0;

    #########################################################
    ###### FILTER, BUILD HISTOGRAMS - START EVENT LOOP ######
    #########################################################


    # Initial cuts
    f1 = rdf.Filter("! SkipTime( timestamp )", 'check time - selected')

    if time_group_divisor != 0:
        f1 = f1.Filter("! SkipTimeInterval( timestamp, %s, %s )".format(time_group_divisor, time_group_remainder),
                        'time interval')

    # Diagonal cut (L831)
    f2 = f1.Filter("v_L_2_F && v_L_2_N && v_R_2_F && v_R_2_N", 'allDiagonalRPs')

    model = ("h_timestamp_dgn", ";timestamp;rate   (Hz)", 
             int(ROOT.timestamp_bins), ROOT.timestamp_min-0.5, ROOT.timestamp_max+0.5)
    h_timestamp_dgn = f2.Histo1D(model, "timestamp")

    # Not cut for this filter in original code
    f_zerobias = f2.Filter("! ((trigger_bits & 512) != 0)", 'zero_bias_event')

    xs = ["x_L_1_F", "x_L_2_N", "x_L_2_F", "x_R_1_F", "x_R_2_N", "x_R_2_F"]
    ys = ["y_L_1_F", "y_L_2_N", "y_L_2_F", "y_R_1_F", "y_R_2_N", "y_R_2_F"]

    # Apply fine alignment (L 852)
    r2 = f2.Define("h_al", "ApplyFineAlignment( timestamp ,{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})".format(*(xs+ys) )) \
        .Define("h_al_x_L_1_F", "h_al.L_1_F.x") \
        .Define("h_al_x_L_2_N", "h_al.L_2_N.x") \
        .Define("h_al_x_L_2_F", "h_al.L_2_F.x") \
        .Define("h_al_y_L_1_F", "h_al.L_1_F.y") \
        .Define("h_al_y_L_2_N", "h_al.L_2_N.y") \
        .Define("h_al_y_L_2_F", "h_al.L_2_F.y") \
        .Define("h_al_x_R_1_F", "h_al.R_1_F.x") \
        .Define("h_al_x_R_2_N", "h_al.R_2_N.x") \
        .Define("h_al_x_R_2_F", "h_al.R_2_F.x") \
        .Define("h_al_y_R_1_F", "h_al.R_1_F.y") \
        .Define("h_al_y_R_2_N", "h_al.R_2_N.y") \
        .Define("h_al_y_R_2_F", "h_al.R_2_F.y") \

    # fill pre-selection histograms
    al_nosel_models = [
        ("h_y_L_1_F_vs_x_L_1_F_al_nosel", ";x^{L,1,F};y^{L,1,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_al_nosel", ";x^{L,2,N};y^{L,2,N}", 150, -15., 15., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_al_nosel", ";x^{L,2,F};y^{L,2,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_al_nosel", ";x^{R,1,F};y^{R,1,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_al_nosel", ";x^{R,2,N};y^{R,2,N}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_al_nosel", ";x^{R,2,F};y^{R,2,F}", 150, -15., 15., 300, -30., +30.)
    ]

    h_y_L_1_F_vs_x_L_1_F_al_nosel = r2.Histo2D(al_nosel_models[0], "h_al_x_L_1_F", "h_al_y_L_1_F")
    h_y_L_2_N_vs_x_L_2_N_al_nosel = r2.Histo2D(al_nosel_models[1], "h_al_x_L_2_N", "h_al_y_L_2_N")
    h_y_L_2_F_vs_x_L_2_F_al_nosel = r2.Histo2D(al_nosel_models[2], "h_al_x_L_2_F", "h_al_y_L_2_F")
    h_y_R_1_F_vs_x_R_1_F_al_nosel = r2.Histo2D(al_nosel_models[3], "h_al_x_R_1_F", "h_al_y_R_1_F")
    h_y_R_2_N_vs_x_R_2_N_al_nosel = r2.Histo2D(al_nosel_models[4], "h_al_x_R_2_N", "h_al_y_R_2_N")
    h_y_R_2_F_vs_x_R_2_F_al_nosel = r2.Histo2D(al_nosel_models[5], "h_al_x_R_2_F", "h_al_y_R_2_F")

    # run reconstruction 
    ### kinematics struct
    r3 = r2.Define("kinematics", 'DoReconstruction( h_al )')

    r4 =  r3.Define("k_th_x_R",        "kinematics.th_x_R")   \
            .Define("k_th_y_R",        "kinematics.th_y_R")   \
            .Define("k_th_x_L",        "kinematics.th_x_L")   \
            .Define("k_th_y_L",        "kinematics.th_y_L")   \
            .Define("k_th_x",          "kinematics.th_x")     \
            .Define("k_th_y",          "kinematics.th_y")     \
            .Define("minus_k_th_y",    "- kinematics.th_y")   \
            .Define("k_vtx_x",         "kinematics.vtx_x")    \
            .Define("k_vtx_x_L",       "kinematics.vtx_x_L")  \
            .Define("k_vtx_x_R",       "kinematics.vtx_x_R")  \
            .Define("k_vtx_y",         "kinematics.vtx_y")    \
            .Define("k_vtx_y_L",       "kinematics.vtx_y_L")  \
            .Define("k_vtx_y_R",       "kinematics.vtx_y_R")  \
            .Define("k_th_y_L_F",      "kinematics.th_y_L_F") \
            .Define("k_th_y_L_N",      "kinematics.th_y_L_N") \
            .Define("k_th_y_R_F",      "kinematics.th_y_R_F") \
            .Define("k_th_y_R_N",      "kinematics.th_y_R_N") \
            .Define("k_th_x_diffLR",   "kinematics.th_x_R - kinematics.th_x_L")     \
            .Define("k_th_y_diffLR",   "kinematics.th_y_R - kinematics.th_y_L")     \
            .Define("k_th_x_diffLF",   "kinematics.th_x_L - kinematics.th_x")       \
            .Define("k_th_x_diffRF",   "kinematics.th_x_R - kinematics.th_x")       \
            .Define("k_th_y_L_diffNF", "kinematics.th_y_L_F - kinematics.th_y_L_N") \
            .Define("k_th_y_R_diffNF", "kinematics.th_y_R_F - kinematics.th_y_R_N") \
            .Define("k_vtx_x_diffLR",  "kinematics.vtx_x_R - kinematics.vtx_x_L")   \
            .Define("k_vtx_y_diffLR",  "kinematics.vtx_y_R - kinematics.vtx_y_L")   \
            .Define("k_t",             "kinematics.t")                              \
            .Define("k_th",            "kinematics.th")                             \
            .Define("k_phi",           "kinematics.phi")

    # cut evaluation
    r5 = r4.Define("cutdata", "EvaluateCutsRDF( h_al, kinematics, anal )")

    # Elastic cut
    f4 = r5.Filter("cutdata.select", "elastic cut")

    # Fill raw histograms
    model = ("h_timestamp_sel", ";timestamp;rate   (Hz)", 
             int(ROOT.timestamp_bins), ROOT.timestamp_min-0.5, ROOT.timestamp_max+0.5)
    h_timestamp_sel = f4.Histo1D(model, "timestamp");

    # fill histograms
    noal_sel_models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_y_L_1_F_vs_x_L_1_F_noal_sel", ";x^{L,1,F};y^{L,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_noal_sel", ";x^{L,2,N};y^{L,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_noal_sel", ";x^{L,2,F};y^{L,2,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_noal_sel", ";x^{R,1,F};y^{R,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_noal_sel", ";x^{R,2,N};y^{R,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_noal_sel", ";x^{R,2,F};y^{R,2,F}", 100, -3., +3., 300, -30., +30.)
    ])

    h_y_L_1_F_vs_x_L_1_F_noal_sel = f4.Histo2D(noal_sel_models[0], "x_L_1_F", "y_L_1_F")
    h_y_L_2_N_vs_x_L_2_N_noal_sel = f4.Histo2D(noal_sel_models[1], "x_L_2_N", "y_L_2_N")
    h_y_L_2_F_vs_x_L_2_F_noal_sel = f4.Histo2D(noal_sel_models[2], "x_L_2_F", "y_L_2_F")
    h_y_R_1_F_vs_x_R_1_F_noal_sel = f4.Histo2D(noal_sel_models[3], "x_R_1_F", "y_R_1_F")
    h_y_R_2_N_vs_x_R_2_N_noal_sel = f4.Histo2D(noal_sel_models[4], "x_R_2_N", "y_R_2_N")
    h_y_R_2_F_vs_x_R_2_F_noal_sel = f4.Histo2D(noal_sel_models[5], "x_R_2_F", "y_R_2_F")

    al_sel_models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_y_L_1_F_vs_x_L_1_F_al_sel", ";x^{L,1,F};y^{L,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_al_sel", ";x^{L,2,N};y^{L,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_al_sel", ";x^{L,2,F};y^{L,2,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_al_sel", ";x^{R,1,F};y^{R,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_al_sel", ";x^{R,2,N};y^{R,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_al_sel", ";x^{R,2,F};y^{R,2,F}", 100, -3., +3., 300, -30., +30.)
    ])

    h_y_L_1_F_vs_x_L_1_F_al_sel = f4.Histo2D(al_sel_models[0], "h_al_x_L_1_F", "h_al_y_L_1_F")
    h_y_L_2_N_vs_x_L_2_N_al_sel = f4.Histo2D(al_sel_models[1], "h_al_x_L_2_N", "h_al_y_L_2_N")
    h_y_L_2_F_vs_x_L_2_F_al_sel = f4.Histo2D(al_sel_models[2], "h_al_x_L_2_F", "h_al_y_L_2_F")
    h_y_R_1_F_vs_x_R_1_F_al_sel = f4.Histo2D(al_sel_models[3], "h_al_x_R_1_F", "h_al_y_R_1_F")
    h_y_R_2_N_vs_x_R_2_N_al_sel = f4.Histo2D(al_sel_models[4], "h_al_x_R_2_N", "h_al_y_R_2_N")
    h_y_R_2_F_vs_x_R_2_F_al_sel = f4.Histo2D(al_sel_models[5], "h_al_x_R_2_F", "h_al_y_R_2_F")

    # Line 1157 (k.th_x_R - k.th_x_L)
    #           (k.th_y_R - k.th_y_L)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("th_x_diffLR", ";#theta_{x}^{R} - #theta_{x}^{L}", 1000, -500E-6, +500E-6),
        ("th_y_diffLR", ";#theta_{y}^{R} - #theta_{y}^{L}", 500, -50E-6, +50E-6)
    ])
    th_x_diffLR = f4.Histo1D(models[0], "k_th_x_diffLR")
    th_y_diffLR = f4.Histo1D(models[1], "k_th_y_diffLR")

    # Line 1160 (k.th_x_L - k.th_x)
    #           (k.th_x_R - k.th_x)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("th_x_diffLF", ";#theta_{x}^{L} - #theta_{x}", 400, -200E-6, +200E-6),
        ("th_x_diffRF", ";#theta_{x}^{R} - #theta_{x}", 400, -200E-6, +200E-6)
    ])
    th_x_diffLF = f4.Histo1D(models[0], "k_th_x_diffLF")
    th_x_diffRF = f4.Histo1D(models[1], "k_th_x_diffRF")

    # Line 1163 (k.th_x, k.th_x_R - k.th_x_L)
    #           (k.th_y, k.th_y_R - k.th_y_L)
    #           (k.vtx_x, k.th_x_R - k.th_x_L)
    models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_th_x_diffLR_vs_th_x", ";#theta_{x};#theta_{x}^{R} - #theta_{x}^{L}", 100, -300E-6, +300E-6, 120, -120E-6, +120E-6),
        ("h_th_y_diffLR_vs_th_y", ";#theta_{y};#theta_{y}^{R} - #theta_{y}^{L}", 100, -500E-6, +500E-6, 120, -120E-6, +120E-6),
        ("h_th_x_diffLR_vs_vtx_x", ";vtx_{x};#theta_{x}^{R} - #theta_{x}^{L}", 100, -300E-3, +300E-3, 120, -120E-6, +120E-6)
    ])
    h_th_x_diffLR_vs_th_x  = f4.Histo2D(models[0], "k_th_x", "k_th_x_diffLR")
    h_th_y_diffLR_vs_th_y  = f4.Histo2D(models[1], "k_th_y", "k_th_y_diffLR")
    h_th_x_diffLR_vs_vtx_x = f4.Histo2D(models[2], "k_vtx_x", "k_th_x_diffLR")


    # Line 1188 (k.th_x_L, k.th_y_L)
    #           (k.th_x_R, k.th_y_R)
    #           (k.th_x, k.th_y)
    models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_th_y_L_vs_th_x_L", ";#theta_{x}^{L};#theta_{y}^{L}", 100, -115E-6, +11E-5, 100, 22E-6, +102E-6),
        ("h_th_y_R_vs_th_x_R", ";#theta_{x}^{R};#theta_{y}^{R}", 100, -125E-6, +12E-5, 100, 27E-6, +102E-6),
        ("h_th_y_vs_th_x", ";#theta_{x};#theta_{y}", 100, -300E-6, +300E-6, 100, -150E-6, +150E-6)
    ])
    h_th_y_L_vs_th_x_L = f4.Histo2D(models[0], "k_th_x_L", "k_th_y_L")
    h_th_y_R_vs_th_x_R = f4.Histo2D(models[1], "k_th_x_R", "k_th_y_R")
    h_th_y_vs_th_x     = f4.Histo2D(models[2], "k_th_x", "k_th_y")

    # Line 1199 (k.th_y_R, k.th_y_L)
    model = ROOT.ROOT.RDF.TH2DModel("h_th_y_L_vs_th_y_R", ";#theta_{y}^{R};#theta_{y}^{L}",
                                    300, -150E-6, +150E-6, 300, -150E-6, +150E-6)
    h_th_y_L_vs_th_y_R = f4.Histo2D(model, "k_th_y_R", "k_th_y_L")

    # Line 1203: (k.th_x)
    #            (k.th_y)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_th_x", ";#theta_{x}", 250, -500E-6, +500E-6),
        ("h_th_y", ";#theta_{y}", 250, -500E-6, +500E-6)
    ])
    h_th_x     = f4.Histo1D(models[0], "k_th_x")
    h_th_y     = f4.Histo1D(models[1], "k_th_y")

    # Line 1205: (-k.th_y)
    model = ("h_th_y_flipped", ";#theta_{y}", 250, -500E-6, +500E-6)
    h_th_y_flipped = f4.Histo1D(model, "minus_k_th_y")

    # Line 1207: (k.th_x_L)
    #            (k.th_x_R)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_th_x_L", ";#theta_{x}^{L}", 250, -500E-6, +500E-6),
        ("h_th_x_R", ";#theta_{x}^{R}", 250, -500E-6, +500E-6)
    ])
    h_th_x_L   = f4.Histo1D(models[0], "k_th_x_L")
    h_th_x_R   = f4.Histo1D(models[1], "k_th_x_R")

    # Line 1210: (k.th_y_L)
    #            (k.th_y_R)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_th_y_L", ";#theta_{y}^{L}", 250, -500E-6, +500E-6),
        ("h_th_y_R", ";#theta_{y}^{R}", 250, -500E-6, +500E-6)
    ])
    h_th_y_L   = f4.Histo1D(models[0], "k_th_y_L")
    h_th_y_R   = f4.Histo1D(models[1], "k_th_y_R")

    # Line 1213: (k.th_y_L_F)
    #            (k.th_y_L_N)
    #            (k.th_y_R_N)
    #            (k.th_y_R_F)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_th_y_L_F", ";#theta_{y}^{L_F}", 250, -500E-6, +500E-6),
        ("h_th_y_L_N", ";#theta_{y}^{L_N}", 250, -500E-6, +500E-6),
        ("h_th_y_R_N", ";#theta_{y}^{R_N}", 250, -500E-6, +500E-6),
        ("h_th_y_R_F", ";#theta_{y}^{R_F}", 250, -500E-6, +500E-6)
    ])
    h_th_y_L_F = f4.Histo1D(models[0], "k_th_y_L_F")
    h_th_y_L_N = f4.Histo1D(models[1], "k_th_y_L_N")
    h_th_y_R_N = f4.Histo1D(models[2], "k_th_y_R_N")
    h_th_y_R_F = f4.Histo1D(models[3], "k_th_y_R_F")


    # fill vertex histograms

    # Line 1220 (k.vtx_x)
    #           (k.vtx_x_L)
    #           (k.vtx_x_R)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_vtx_x", ";x^{*}", 100, -0.5, +0.5),
        ("h_vtx_x_L", ";x^{*,L}", 100, -0.5, +0.5),
        ("h_vtx_x_R", ";x^{*,R}", 100, -0.5, +0.5)
    ])
    h_vtx_x    = f4.Histo1D(models[0], "k_vtx_x")
    h_vtx_x_L  = f4.Histo1D(models[1], "k_vtx_x_L")
    h_vtx_x_R  = f4.Histo1D(models[2], "k_vtx_x_R")

    # Line 1224 (k.vtx_y)
    #           (k.vtx_y_L)
    #           (k.vtx_y_R)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_vtx_y", ";y^{*}", 100, -0.5, +0.5),
        ("h_vtx_y_L", ";y^{*,L}", 100, -0.5, +0.5),
        ("h_vtx_y_R", ";y^{*,R}", 100, -0.5, +0.5)
    ])
    h_vtx_y    = f4.Histo1D(models[0], "k_vtx_y")
    h_vtx_y_L  = f4.Histo1D(models[1], "k_vtx_y_L")
    h_vtx_y_R  = f4.Histo1D(models[2], "k_vtx_y_R")

    # Line 1228:
    #            (k.vtx_x_R, k.vtx_x_L)
    #            (k.vtx_y_R, k.vtx_y_L)
    models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_vtx_x_L_vs_vtx_x_R", ";x^{*,R};x^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5),
        ("h_vtx_y_L_vs_vtx_y_R", ";y^{*,R};y^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5)
    ])
    h_vtx_x_L_vs_vtx_x_R = f4.Histo2D(models[0], "k_vtx_x_R", "k_vtx_x_L")
    h_vtx_y_L_vs_vtx_y_R = f4.Histo2D(models[1], "k_vtx_y_R", "k_vtx_y_L")

    # Line 1231:
    #            (k.th_x_L, k.vtx_x_L)
    #            (k.th_x_R, k.vtx_x_R)
    #            (k.th_y_L, k.vtx_y_L)
    #            (k.th_y_R, k.vtx_y_R)
    models = map(ROOT.ROOT.RDF.TH2DModel, [
        ("h_vtx_x_L_vs_th_x_L", ";#theta_{x}^{L};x^{*,L}", 100, -600E-6, +600E-6, 100, -0.5, +0.5),
        ("h_vtx_x_R_vs_th_x_R", ";#theta_{x}^{R};x^{*,R}", 100, -600E-6, +600E-6, 100, -0.5, +0.5),
        ("h_vtx_y_L_vs_th_y_L", ";#theta_{y}^{L};y^{*,L}", 100, -600E-6, +600E-6, 100, -0.5, +0.5),
        ("h_vtx_y_R_vs_th_y_R", ";#theta_{y}^{R};y^{*,R}", 100, -600E-6, +600E-6, 100, -0.5, +0.5)
    ])
    h_vtx_x_L_vs_th_x_L = f4.Histo2D(models[0], "k_th_x_L", "k_vtx_x_L")
    h_vtx_x_R_vs_th_x_R = f4.Histo2D(models[1], "k_th_x_R", "k_vtx_x_R")
    h_vtx_y_L_vs_th_y_L = f4.Histo2D(models[2], "k_th_y_L", "k_vtx_y_L")
    h_vtx_y_R_vs_th_y_R = f4.Histo2D(models[3], "k_th_y_R", "k_vtx_y_R")

    # Line 1236:
    #           (k.vtx_x_R - k.vtx_x_L)
    #           (k.vtx_y_R - k.vtx_y_L)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_vtx_x_diffLR", ";x^{*,R} - x^{*,L}", 100, -0.5, +0.5),
        ("h_vtx_y_diffLR", ";y^{*,R} - y^{*,L}", 100, -0.5, +0.5)
    ])
    h_vtx_x_diffLR = f4.Histo1D(models[0], "k_vtx_x_diffLR");
    h_vtx_y_diffLR = f4.Histo1D(models[1], "k_vtx_y_diffLR");

    # Line 1239:
    #           (k.th_x, k.vtx_x_R - k.vtx_x_L)
    #           (k.th_y, k.vtx_y_R - k.vtx_y_L)
    models = map(ROOT.ROOT.RDF.TH1DModel, [
        ("h_vtx_x_diffLR", ";x^{*,R} - x^{*,L}", 100, -0.5, +0.5),
        ("h_vtx_y_diffLR", ";y^{*,R} - y^{*,L}", 100, -0.5, +0.5)
    ])
    h_vtx_x_diffLR_vs_th_x = f4.Histo1D(models[0], "k_th_x", "k_vtx_x_diffLR");
    h_vtx_y_diffLR_vs_th_y = f4.Histo1D(models[1], "k_th_y", "k_vtx_y_diffLR");

    # Line 1245:
    #           (k.vtx_x_R, k.vtx_x_R - k.vtx_x_L)
    #           (k.vtx_y_R, k.vtx_y_R - k.vtx_y_L)
    models = map(ROOT.ROOT.RDF.TH2DModel,[
        ("h_vtx_x_diffLR_vs_vtx_x_R", ";x^{*,R};x^{*,R} - x^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5),
        ("h_vtx_y_diffLR_vs_vtx_y_R", ";y^{*,R};y^{*,R} - y^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5)
    ])
    h_vtx_x_diffLR_vs_vtx_x_R = f4.Histo2D(models[0], "k_vtx_x_R", "k_vtx_y_diffLR");
    h_vtx_y_diffLR_vs_vtx_y_R = f4.Histo2D(models[1], "k_vtx_y_R", "k_vtx_y_diffLR");

    # Define normalization and norm_corr colums
    r6 = f4.Define("norm_corr",     "getNorm_corr( timestamp )" ) \
        .Define("normalization", "getNormalization( norm_corr )")

    
    # calculate acceptance divergence correction
    r7 = r6.Define("correction", "CalculateAcceptanceCorrectionsRDF( th_y_sign, kinematics, anal )") \
        .Define("corr",       "correction.corr") \
        .Define("div_corr",   "correction.div_corr") \
        .Define("one",        "One()")

    # Line 1406
    for bi in binnings:
        bis = binning_setup[bi]

        model = ROOT.RDF.TH1DModel("h_t_Nev_before", ";|t|;events per bin", bis.N_bins, bis.bin_edges)
        bh_t_Nev_before[bi] = r7.Histo1D(model, "k_t", "one");

        model = ROOT.RDF.TH1DModel("h_t_before", ";|t|", bis.N_bins, bis.bin_edges)
        bh_t_before[bi] = r7.Histo1D(model, "k_t", "one");


    # Line 1412
    model = ROOT.RDF.TH2DModel("h_th_y_vs_th_x_before", ";#theta_{x};#theta_{y}", 150, -300E-6, +300E-6, 150, -150E-6, +150E-6)
    h_th_y_vs_th_x_before = r7.Histo2D(model, "k_th_x", "k_th_y", "one");

    # Line 1414
    # Filter skip
    f5 = r7.Filter("! correction.skip", "acceptance correction")

    # Line 1429
    for bi in binnings:
        bis = binning_setup[bi]

        model = ROOT.RDF.TH1DModel("h_t_Nev_after_no_corr", ";|t|;events per bin", bis.N_bins, bis.bin_edges)
        bh_t_Nev_after_no_corr[bi] = f5.Histo1D(model, "k_t", "one");

        model = ROOT.RDF.TH1DModel("h_t_after_no_corr", ";|t|", bis.N_bins, bis.bin_edges)
        bh_t_after_no_corr[bi] = f5.Histo1D(model, "k_t", "one");

        model = ROOT.RDF.TH1DModel("h_t_after", ";|t|", bis.N_bins, bis.bin_edges)
        bh_t_after[bi] = f5.Histo1D(model, "k_t", "corr");

    # Line 1435
    model = ROOT.RDF.TH2DModel("h_th_y_vs_th_x_after", ";#theta_{x};#theta_{y}", 150, -300E-6, +300E-6, 150, -150E-6, +150E-6);
    h_th_y_vs_th_x_after = f5.Histo2D(model, "k_th_x", "k_th_y", "div_corr");

    # Line 1435
    model = ROOT.RDF.TH2DModel("h_th_vs_phi_after", ";#phi;#theta", 50, -M_PI, +M_PI, 50, 150E-6, 550E-6);
    h_th_vs_phi_after = f5.Histo2D(model, "k_phi", "k_th", "div_corr");

    # Line 1441
    # apply normalization
    model = ROOT.ROOT.RDF.TH1DModel("h_t_normalized", ";|t|",128, 0., 4.)
    bh_t_normalized_ob_1_30_02 = f5.Define("corr_norm", "corr * normalization") \
                                .Histo1D(model, "k_t", "corr_norm")

    # Line 1445
    model = ROOT.RDF.TH2DModel("h_th_y_vs_th_x_normalized", ";#theta_{x};#theta_{y}", 150, -600E-6, +600E-6, 150, -600E-6, +600E-6);
    h_th_y_vs_th_x_normalized = f5.Define("div_corr_norm", "correction.div_corr * normalization") \
                                .Histo2D(model, "k_th_x", "k_th_y", "div_corr_norm");


    # normalize histograms
    for bi in binnings:
        bh_t_before[bi].Scale(1., "width");
        bh_t_after_no_corr[bi].Scale(1., "width");
        bh_t_after[bi].Scale(1., "width");
        
    hlist.append(h_timestamp_dgn) #0
    hlist.append(h_y_L_1_F_vs_x_L_1_F_al_nosel) #1
    hlist.append(h_y_L_2_N_vs_x_L_2_N_al_nosel)
    hlist.append(h_y_L_2_F_vs_x_L_2_F_al_nosel)
    hlist.append(h_y_R_1_F_vs_x_R_1_F_al_nosel)
    hlist.append(h_y_R_2_N_vs_x_R_2_N_al_nosel)
    hlist.append(h_y_R_2_F_vs_x_R_2_F_al_nosel) #6
    hlist.append(h_timestamp_sel) #7
    hlist.append(h_y_L_1_F_vs_x_L_1_F_noal_sel) #8
    hlist.append(h_y_L_2_N_vs_x_L_2_N_noal_sel)
    hlist.append(h_y_L_2_F_vs_x_L_2_F_noal_sel)
    hlist.append(h_y_R_1_F_vs_x_R_1_F_noal_sel)
    hlist.append(h_y_R_2_N_vs_x_R_2_N_noal_sel)
    hlist.append(h_y_R_2_F_vs_x_R_2_F_noal_sel) #13
    hlist.append(h_y_L_1_F_vs_x_L_1_F_al_sel) #14
    hlist.append(h_y_L_2_N_vs_x_L_2_N_al_sel)
    hlist.append(h_y_L_2_F_vs_x_L_2_F_al_sel)
    hlist.append(h_y_R_1_F_vs_x_R_1_F_al_sel)
    hlist.append(h_y_R_2_N_vs_x_R_2_N_al_sel)
    hlist.append(h_y_R_2_F_vs_x_R_2_F_al_sel) #19
    hlist.append(th_x_diffLR) #20
    hlist.append(th_y_diffLR)
    hlist.append(th_x_diffLF) #22
    hlist.append(th_x_diffRF)
    hlist.append(h_th_x_diffLR_vs_th_x) #24
    hlist.append(h_th_y_diffLR_vs_th_y)
    hlist.append(h_th_x_diffLR_vs_vtx_x)
    hlist.append(h_th_y_L_vs_th_x_L) #27
    hlist.append(h_th_y_R_vs_th_x_R)
    hlist.append(h_th_y_vs_th_x)
    hlist.append(h_th_y_L_vs_th_y_R) #30
    hlist.append(h_th_x) #31
    hlist.append(h_th_y)
    hlist.append(h_th_y_flipped) #33
    hlist.append(h_th_x_L) #34
    hlist.append(h_th_x_R)
    hlist.append(h_th_y_L) #36
    hlist.append(h_th_y_R)
    hlist.append(h_th_y_L_F) #38
    hlist.append(h_th_y_L_N)
    hlist.append(h_th_y_R_N)
    hlist.append(h_th_y_R_F) #41
    hlist.append(h_vtx_x) #42
    hlist.append(h_vtx_x_L)
    hlist.append(h_vtx_x_R) #44
    hlist.append(h_vtx_y) #45
    hlist.append(h_vtx_y_L)
    hlist.append(h_vtx_y_R)
    hlist.append(h_vtx_x_L_vs_vtx_x_R) #48
    hlist.append(h_vtx_y_L_vs_vtx_y_R)
    hlist.append(h_vtx_x_L_vs_th_x_L) #50
    hlist.append(h_vtx_x_R_vs_th_x_R)
    hlist.append(h_vtx_y_L_vs_th_y_L)
    hlist.append(h_vtx_y_R_vs_th_y_R) #53
    hlist.append(h_vtx_x_diffLR) #54
    hlist.append(h_vtx_y_diffLR) 
    hlist.append(h_vtx_x_diffLR_vs_th_x) #56
    hlist.append(h_vtx_y_diffLR_vs_th_y)
    hlist.append(h_vtx_x_diffLR_vs_vtx_x_R) #58
    hlist.append(h_vtx_y_diffLR_vs_vtx_y_R)

    for bi in binnings:
        hlist.append(bh_t_Nev_before[bi]) # 60, 62, 64
        hlist.append(bh_t_before[bi]) #61, 63, 65
    hlist.append(h_th_y_vs_th_x_before) # 60 + len(binnings) * 2 (should be 66 here)
    for bi in binnings:
        hlist.append(bh_t_Nev_after_no_corr[bi]) # 61 + len(binnings) * 2 (should be 67 here)
        hlist.append(bh_t_after_no_corr[bi])
        hlist.append(bh_t_after[bi])
    hlist.append(h_th_y_vs_th_x_after) # 60 + len(binnings)*5 + 1
    hlist.append(h_th_vs_phi_after) 
    hlist.append(bh_t_normalized_ob_1_30_02)
    hlist.append(h_th_y_vs_th_x_normalized)    

    return []

As you can see, our code is basically following the same structure as the original version with two main functions: `distill` and `distributions`. Unlike the reference code, the creation of an intermediate ROOT file with a reduced dataset is no longer needed since running in parallel will shorten the execution time.

In order to go distributed, we need to encapsulate our analysis in a function that receives a `RDataFrame`. Thus, we define the `fillHist` function, whose body corresponds to the `RDataFrame` analysis we already saw in the local execution example. Note how the function returns the list of histogram produced during the analysis.

As more than one histogram is produced during `distributions`, we need to append all of them to a list so they can be reachable at the reduction phase.

In [6]:
def fillHist(rdf):
    distilled = distill(rdf)
    return distributions(distilled)

In the current implementation, a second function is needed to merge partial results - in this case, a **list** of partially filled histograms coming from the distributed parallelization. This requirement will be removed for the programming model to be simpler, and the implementation will automatically take care of the merge.

In [7]:
def mergeHist(l1, l2):
    [first.Add(second) for (first, second) in zip(l1, l2)]
    return l1

The next cell launches the distributed computation with Spark, where a `DistTree` is constructed from three parameters:
- List of files of our dataset
- Tree name
- Desired number of partitions of our dataset (will potentially determine the amount of parallelism we will achieve)

Once the `DistTree` is created, we invoke the `ProcessAndMerge` method on it to trigger the distributed processing. The two arguments of this call are the two functions defined previously.

In [8]:
%%time 

# Name of the ROOT Tree
input_ntuple_name = "TotemNtuple"


selected_ds = ['DS1', 'DS2', 'DS3', 'DS4', 'DS5', 'DS6', 'DS7']

source_files = ["input_files_{}.txt".format(ds) for ds in selected_ds]

# Remote path in EOS
prefixes            = ["root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/{}/".format(DS[ds]) for ds in selected_ds]
input_files       = [prefixes[i] + line.rstrip('\r\n') for i in range(len(source_files)) for line in open(source_files[i]) ]

# All datasets
dTree = DistTree(input_files,
                 treename = "TotemNtuple",
                 npartitions = 512)

histos = dTree.ProcessAndMerge(fillHist, mergeHist)

CPU times: user 3 s, sys: 1.26 s, total: 4.26 s
Wall time: 5min 43s


At this point, if the everything run successfully, the analysis will be completed. 

It is worthwhile to mention that from this cell on, every execution will be local again. This is interesting since now we are able to select which parts of our analysis will run in local or distributed on the same notebook, thus reducing execution times while keeping an interactive session.

### Visualizing results


As a result of `ProccessAndMerge` a list of already merged histograms is stored in `histos`. 

In [9]:
# 1 partition
%jsroot on
c = ROOT.TCanvas();
histos[75].Draw("colz")
ROOT.gPad.SetLogy(); # Set Y-axis as logarithmic scale
c.Draw()

### Saving results

Finally, we can save the histograms on a ROOT file.

In [16]:
histos[21].Scale(1., "width");
histos[20].Scale(1., "width");

In [17]:
# save histograms
c = ROOT.TCanvas();
outF = ROOT.TFile.Open("distributions_" + selected_diagonal + ".root", "recreate");

In [18]:
ROOT.gDirectory = outF.mkdir("metadata");
c = ROOT.TCanvas("rate cmp");
histos[0].Draw();
#h_timestamp_B0.SetLineColor(4);
#h_timestamp_B0.Draw("sames");
histos[7].SetLineColor(2);
histos[7].Draw("sames");
c.Write();

In [19]:
# Line 1700
hitDistDir = outF.mkdir("hit distributions");
ROOT.gDirectory = hitDistDir.mkdir("vertical, aligned, before selection");
histos[3].Write();
histos[2].Write();
histos[1].Write();
histos[4].Write();
histos[5].Write();
histos[6].Write();

In [20]:
ROOT.gDirectory = hitDistDir.mkdir("vertical, not aligned, after selection");
histos[10].Write();
histos[9].Write();
histos[8].Write();
histos[11].Write();
histos[12].Write();
histos[13].Write();

In [21]:
ROOT.gDirectory = hitDistDir.mkdir("vertical, aligned, after selection");
histos[14].Write();
histos[15].Write();
histos[16].Write();
histos[17].Write();
histos[18].Write();
histos[19].Write();

In [22]:
ROOT.gDirectory = outF.mkdir("selected - hits");
ROOT.gDirectory = outF.mkdir("selected - angles");

histos[20].Sumw2(); histos[20].Write()
histos[21].Sumw2(); histos[21].Write()

histos[22].Sumw2(); histos[22].Write()
histos[23].Sumw2(); histos[23].Write()

histos[24].Write();
histos[25].Write();
histos[26].Write();

histos[27].Write();
histos[28].Write();
histos[29].Write();

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [23]:
c = ROOT.TCanvas();
c.SetLogz(1);
c.ToggleEventStatus();
c.SetCrosshair(1);
histos[30].Draw("colz");
#g_th_y_L_vs_th_y_R.Draw("p");
c.Write("canvas_th_y_L_vs_th_y_R");

histos[31].SetLineColor(1); histos[31].Write();
histos[32].SetLineColor(1); histos[32].Write();
histos[33].SetLineColor(1); histos[33].Write();

histos[34].SetLineColor(2); histos[34].Write();
histos[35].SetLineColor(4); histos[35].Write();

histos[36].SetLineColor(2); histos[36].Write();
histos[37].SetLineColor(4); histos[37].Write();

histos[38].SetLineColor(2); histos[38].Write();
histos[39].SetLineColor(6); histos[39].Write();
histos[40].SetLineColor(4); histos[40].Write();
histos[41].SetLineColor(7); histos[41].Write();

In [24]:
ROOT.gDirectory = outF.mkdir("selected - vertex");
histos[42].SetLineColor(1); histos[42].Write();
histos[43].SetLineColor(2); histos[43].Write();
histos[44].SetLineColor(4); histos[44].Write();

histos[45].SetLineColor(1); histos[45].Write();
histos[46].SetLineColor(2); histos[46].Write();
histos[47].SetLineColor(4); histos[47].Write();

histos[48].Write();
histos[49].Write();

histos[50].Write();
histos[51].Write();
histos[52].Write();
histos[53].Write();

histos[54].Sumw2(); histos[54].Write();
histos[55].Sumw2(); histos[55].Write();

histos[56].Write();
histos[57].Write();

histos[58].Write();
histos[59].Write();

In [25]:
accDir = outF.mkdir("acceptance correction");
i = 0
for bi in binnings:
    ROOT.gDirectory = accDir.mkdir(bi);
    #bh_t_Nev_before[bi].Sumw2();        bh_t_Nev_before[bi].Write();
    histos[60+i*2].Sumw2();        histos[60+i*2].Write();
    #bh_t_Nev_after_no_corr[bi].Sumw2(); bh_t_Nev_after_no_corr[bi].Write();
    histos[61+len(binnings)*2 + i*3].Sumw2(); histos[61+len(binnings)*2 + i*3].Write();
    #bh_t_before[bi].Sumw2();            bh_t_before[bi].Write();
    histos[61+i*2].Sumw2();            histos[61+i*2].Write();
    #bh_t_after_no_corr[bi].Sumw2();     bh_t_after_no_corr[bi].Write();
    histos[62+len(binnings)*2 + i*3].Sumw2();     histos[62+len(binnings)*2 + i*3].Write();
    #bh_t_after[bi].Sumw2();             bh_t_after[bi].Write();
    histos[63+len(binnings)*2 + i*3].Sumw2();             histos[63+len(binnings)*2 + i*3].Write();
    i += 1

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [26]:
ROOT.gDirectory = accDir;

histos[60 + len(binnings)*2].Sumw2();     histos[60 + len(binnings)*2].Write();
histos[60 + len(binnings)*5 + 1].Sumw2(); histos[60 + len(binnings)*5 + 1].Write();
histos[60 + len(binnings)*5 + 2].Sumw2(); histos[60 + len(binnings)*5 + 2].Write();

normDir = outF.mkdir("normalization");


Warning in <TH2D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH2D::Sumw2>: Sum of squares of weights structure already created
